<a href="https://colab.research.google.com/github/bhuvnk/END2.0/blob/main/Session%207%20-%20Second%20Hands-on%20Encoder-Decoder/Part2_Seq2Seq/3_AMbig_QA_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jun 25 20:51:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Import Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.data import Field, BucketIterator,TabularDataset

import spacy
import numpy as np
import pandas as pd
import glob

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Use GPU as device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ", device)

Device:  cuda


In [3]:
%%bash
python -m spacy download en 

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


# Data Preprocessing

In [4]:
# !wget https://nlp.cs.washington.edu/ambigqa/data/ambignq.zip
# !unzip -q 'ambignq.zip'

In [5]:
!wget https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip
!unzip -q 'ambignq_light.zip'

--2021-06-25 20:51:44--  https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061383 (1.0M) [application/zip]
Saving to: ‘ambignq_light.zip’

ambignq_light.zip   100%[===================>]   1.01M  1.17MB/s    in 0.9s    

2021-06-25 20:51:46 (1.17 MB/s) - ‘ambignq_light.zip’ saved [1061383/1061383]



## Function to read data Clean 

Lazy parsing only for the `ambignq_light` dataset.

In [6]:
import pandas as pd
def clean_df_from_json(json_file):
  df = pd.read_json(json_file)

  ## Get the answers for outside questions
  df['answer'] = [i[0]['answer'][0] if i[0]['type']=='singleAnswer' else False for i in df['annotations']]

  # single answer data
  df_singleAnswer = df[df['answer'] != False]
  df_singleAnswer.drop(columns=['annotations', 'id'], inplace = True)
  # df_singleAnswer.reset_index(drop = True, inplace=True)

  ## parse multipleQAs into a dictionary
  multipleQAs_dict = {'question':[], 'answer':[]}

  for i in df[df['answer'] == False]['annotations']:
    for j in i[0]['qaPairs']:
      multipleQAs_dict['question'].append(j['question'])
      multipleQAs_dict['answer'].append(j['answer'][0])

  ## Create a dataframe out of it
  df_multipleQAs = pd.DataFrame(multipleQAs_dict)

  ## Merge singleAnswer and MultipleQAs together
  final_df = pd.concat([df_multipleQAs, df_singleAnswer])
  final_df.drop_duplicates()
  final_df.reset_index(drop = True, inplace=True)
  return final_df

## Importing train_test df

In [7]:
train_df = clean_df_from_json("train_light.json")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
print(f'Train dimentions are: {train_df.shape} \n')
train_df.head()

Train dimentions are: (19244, 2) 



,question,answer
0,When did the Simpsons first air on television ...,"April 19, 1987"
1,When did the Simpsons first air as a half-hour...,"December 17, 1989"
2,"What is the legal age of marriage, without par...",18 years of age
3,"What is the legal age of marriage, without par...",19
4,"What is the legal age of marriage, without par...",21


In [9]:
dev_df = clean_df_from_json("dev_light.json")
print(f'dev dimentions are: {dev_df.shape} \n')
dev_df.head()

dev dimentions are: (4377, 2) 



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,question,answer
0,Who is the 4th chairman of african union commi...,Moussa Faki
1,Who is the 3rd chairman of african union commi...,Nkosazana Clarice Dlamini-Zuma
2,Who is the 2nd chairman of african union commi...,Jean Ping
3,Who won the Final HoH in the American reality ...,Kaycee Clark
4,Who won the final vote in the British reality ...,Sarah Harding


In [10]:
# Convert to csv file; tabuler dataset can read it nicely
train_df.to_csv('./train.csv')
dev_df.to_csv('./dev.csv')

# Preparing data for Modeling

## Tokenizing

In [11]:
# Tokenizer function
spacy_en = spacy.load('en_core_web_sm') 

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

## Field Variables

In [12]:
SRC = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

# Question and Answer correspond to columns in the .csv file
fields = {'question' : ('src', SRC),
          'answer' : ('trg', TRG)}

## Create Dataset

In [13]:
# qa_data  = TabularDataset(path='Final_data.csv',
#                           format='csv',
#                           fields=fields
#                            )

In [14]:
full_data  = TabularDataset(path='train.csv',
                          format='csv',
                          fields=fields
                           )
test_data  = TabularDataset(path='dev.csv',
                          format='csv',
                          fields=fields
                           )

## Train Valid split 80:20 on the train data

**Note the dev set `here named test set` is kept for final testing only**

In [15]:
train_data, valid_data = full_data.split([0.8, 0.2])

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 15395
Number of validation examples: 3849
Number of testing examples: 4377


## Build Vocab

In [16]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

print(f"Unique tokens in source (question) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (answer) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (question) vocabulary: 7164
Unique tokens in target (answer) vocabulary: 4430


## Iterators

In [17]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=True,
    device = device)

# Seq2Seq

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [19]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

# Model init

In [21]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [22]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7164, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(4430, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=4430, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,597,070 trainable parameters


# Optimizer and Lossfn

In [24]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

# Train and Evaluation Functions

In [25]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [26]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
# Timer as we aint using hbox
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Model Training

In [28]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'AMbig-QA-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 22s
	Train Loss: 5.031 | Train PPL: 153.090
	 Val. Loss: 4.175 |  Val. PPL:  65.062
Epoch: 02 | Time: 0m 22s
	Train Loss: 4.374 | Train PPL:  79.345
	 Val. Loss: 4.012 |  Val. PPL:  55.276
Epoch: 03 | Time: 0m 22s
	Train Loss: 4.169 | Train PPL:  64.620
	 Val. Loss: 3.937 |  Val. PPL:  51.274
Epoch: 04 | Time: 0m 22s
	Train Loss: 4.035 | Train PPL:  56.570
	 Val. Loss: 3.872 |  Val. PPL:  48.052
Epoch: 05 | Time: 0m 22s
	Train Loss: 3.911 | Train PPL:  49.944
	 Val. Loss: 3.882 |  Val. PPL:  48.530
Epoch: 06 | Time: 0m 22s
	Train Loss: 3.806 | Train PPL:  44.992
	 Val. Loss: 3.864 |  Val. PPL:  47.654
Epoch: 07 | Time: 0m 22s
	Train Loss: 3.706 | Train PPL:  40.691
	 Val. Loss: 3.826 |  Val. PPL:  45.888
Epoch: 08 | Time: 0m 22s
	Train Loss: 3.583 | Train PPL:  35.987
	 Val. Loss: 3.848 |  Val. PPL:  46.888
Epoch: 09 | Time: 0m 22s
	Train Loss: 3.482 | Train PPL:  32.522
	 Val. Loss: 3.847 |  Val. PPL:  46.840
Epoch: 10 | Time: 0m 22s
	Train Loss: 3.381 | Train PPL

# Model Testing

In [29]:
model.load_state_dict(torch.load('AMbig-QA-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.766 | Test PPL:  43.218 |
